<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: Юрий Исаков и Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Тема 4. Линейные модели классификации и регрессии
## <center>  Практика. Идентификация пользователя с помощью логистической регрессии

Тут мы воспроизведем парочку бенчмарков нашего соревнования и вдохновимся побить третий бенчмарк, а также остальных участников. Веб-формы для отправки ответов тут не будет, ориентир – [leaderboard](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard) соревнования.

In [104]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV

### 1. Загрузка и преобразование данных
Зарегистрируйтесь на [Kaggle](www.kaggle.com), если вы не сделали этого раньше, зайдите на [страницу](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования и скачайте данные. Первым делом загрузим обучающую и тестовую выборки и посмотрим на данные.

In [2]:
# загрузим обучающую и тестовую выборки
train_df = pd.read_csv('../../data/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/train_sessions.csv',
                       index_col='session_id',
                       parse_dates=True)
test_df = pd.read_csv('../../data/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/test_sessions.csv',
                      index_col='session_id',
                      parse_dates=True)

# приведем колонки time1, ..., time10 к временному формату
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# отсортируем данные по времени
train_df = train_df.sort_values(by='time1')

# посмотрим на заголовок обучающей выборки
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


In [3]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82797 entries, 1 to 82797
Data columns (total 20 columns):
site1     82797 non-null int64
time1     82797 non-null datetime64[ns]
site2     81308 non-null float64
time2     81308 non-null datetime64[ns]
site3     80075 non-null float64
time3     80075 non-null datetime64[ns]
site4     79182 non-null float64
time4     79182 non-null datetime64[ns]
site5     78341 non-null float64
time5     78341 non-null datetime64[ns]
site6     77566 non-null float64
time6     77566 non-null datetime64[ns]
site7     76840 non-null float64
time7     76840 non-null datetime64[ns]
site8     76151 non-null float64
time8     76151 non-null datetime64[ns]
site9     75484 non-null float64
time9     75484 non-null datetime64[ns]
site10    74806 non-null float64
time10    74806 non-null datetime64[ns]
dtypes: datetime64[ns](10), float64(9), int64(1)
memory usage: 13.3 MB


In [4]:
test_df.shape

(82797, 20)

В обучающей выборке содержатся следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - target – целевая переменная, 1 для сессий Элис, 0 для сессий других пользователей
    
Сессии пользователей выделены таким образом, что они не могут быть длиннее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд либо когда сессия заняла по времени более 30 минут.

В таблице встречаются пропущенные значения, это значит, что сессия состоит менее, чем из 10 сайтов. Заменим пропущенные значения нулями и приведем признаки к целому типу. Также загрузим словарь сайтов и посмотрим, как он выглядит:

In [5]:
# приведем колонки site1, ..., site10 к целочисленному формату и заменим пропуски нулями
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# загрузим словарик сайтов
with open(r"../../data/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словарика сайтов
sites_dict_df = pd.DataFrame(list(site_dict.keys()), 
                          index=list(site_dict.values()), 
                          columns=['site'])
print(u'всего сайтов:', sites_dict_df.shape[0])
sites_dict_df.head()

всего сайтов: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


Выделим целевую переменную и объединим выборки, чтобы вместе привести их к разреженному формату.

In [6]:
# наша целевая переменная
y_train = train_df['target']

# объединенная таблица исходных данных
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]

Для самой первой модели будем использовать только посещенные сайты в сессии (но не будем обращать внимание на временные признаки). За таким выбором данных для модели стоит такая идея:  *у Элис есть свои излюбленные сайты, и чем чаще вы видим эти сайты в сессии, тем выше вероятность, что это сессия Элис и наоборот.*

Подготовим данные, из всей таблицы выберем только признаки `site1, site2, ... , site10`. Напомним, что пропущенные значения заменены нулем. Вот как выглядят первые строки таблицы:

In [7]:
# табличка с индексами посещенных сайтов в сессии
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


Сессии представляют собой последовательность индексов сайтов и данные в таком виде неудобны для линейных методов. В соответствии с нашей гипотезой (у Элис есть излюбленные сайты) надо преобразовать эту таблицу таким образом, чтобы каждому возможному сайту соответствовал свой отдельный признак (колонка), а его значение равнялось бы количеству посещений этого сайта в сессии. Это делается в две строчки:

In [8]:
from scipy.sparse import csr_matrix

In [9]:
# csr_matrix?

In [107]:
# последовательность с индексами
sites_flatten = full_sites.values.flatten()
# print(full_sites.values.flatten())

# искомая матрица
full_sites_sparse = csr_matrix(([1] * sites_flatten.shape[0],
                                sites_flatten,
                                range(0, sites_flatten.shape[0] + 10, 10)))[:, 1:]
# print(full_sites_sparse)

In [108]:
# Запишем тестовую и тренировочную выборки по сайтам в файлы
train_df[sites].fillna(0).astype('int').to_csv('train_sessions_text.txt', 
                                               sep=' ', 
                       index=None, header=None)
test_df[sites].fillna(0).astype('int').to_csv('test_sessions_text.txt', 
                                              sep=' ', 
                       index=None, header=None)

In [109]:
cv = CountVectorizer(ngram_range=(1, 3), max_features=50000)
with open('train_sessions_text.txt') as inp_train_file:
    X_train = cv.fit_transform(inp_train_file)
with open('test_sessions_text.txt') as inp_test_file:
    X_test = cv.transform(inp_test_file)

In [111]:
type(X_train) # на выходе - разреженная матрица

scipy.sparse.csr.csr_matrix

In [110]:
X_train.shape, X_test.shape

((253561, 50000), (82797, 50000))

In [11]:
X_train_sparse = full_sites_sparse[:idx_split]
X_test_sparse = full_sites_sparse[idx_split:]

In [12]:
X_train_sparse.shape, y_train.shape

((253561, 48371), (253561,))

In [13]:
X_test_sparse.shape

(82797, 48371)

Еще один плюс использования разреженных матриц в том, что для них имеются специальные реализации как матричных операций, так и алгоритмов машинного обучения, что подчас позволяет ощутимо ускорить операции за счет особенностей структуры данных. Это касается и логистической регрессии. Вот теперь у нас все готово для построения нашей первой модели.

### 2. Построение первой модели

Итак, у нас есть алгоритм и данные для него, построим нашу первую модель, воспользовавшись релизацией [логистической регрессии](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) из пакета `sklearn` с параметрами по умолчанию. Первые 90% данных будем использовать для обучения (обучающая выборка отсортирована по времени), а оставшиеся 10% для проверки качества (validation). 

**Напишите простую функцию, которая будет возвращать качество модели на отложенной выборке, и обучите наш первый классификатор**.

In [102]:
def get_auc_lr_valid(X, y, C=1.0, ratio = 0.9, seed=17):
    '''
    X, y – выборка
    ratio – в каком отношении поделить выборку
    C, seed – коэф-т регуляризации и random_state 
              логистической регрессии
    '''
    
    x_train_len = int(ratio * X.shape[0])
    X_train = X[: x_train_len, :]
    X_test = X[x_train_len :, :]
    
    Y_train = y[: x_train_len]
    Y_test = y[x_train_len :]
    
#     logit = LogisticRegression(n_jobs=-1, C=C, random_state=seed, solver='lbfgs')
    logit = LogisticRegression(n_jobs=-1, C=C, random_state=seed, solver='liblinear')
    
    logit.fit(X_train, Y_train)
    
    pred_valid = logit.predict_proba(X_test)[:, 1]
    return roc_auc_score(Y_test, pred_valid)

**Заведём функцию для проверки числа правильных ответов (как в уроке по Decision tree)**

In [15]:
def get_accuracy_value(X, y, C=1.0, ratio = 0.7, seed=17):
    '''
    X, y – выборка
    ratio – в каком отношении поделить выборку
    C, seed – коэф-т регуляризации и random_state 
              логистической регрессии
    '''
    
    x_train_len = int(ratio * X.shape[0])
    X_train = X[: x_train_len, :]
    X_test = X[x_train_len :, :]
    
    Y_train = y[: x_train_len]
    Y_test = y[x_train_len :]
    
    logit = LogisticRegression(n_jobs=-1, C=C, random_state=seed, solver='lbfgs')
    
    logit.fit(X_train, Y_train)
    
    pred = logit.predict(X_test)
    
    return accuracy_score(Y_test, pred)

**Посмотрите, какой получился ROC AUC на отложенной выборке.**

In [16]:
get_auc_lr_valid(X=X_train_sparse, y=y_train)

0.9197955574958127

Будем считать эту модель нашей первой отправной точкой (baseline). Для построения модели для прогноза на тестовой выборке **необходимо обучить модель заново уже на всей обучающей выборке** (пока наша модель обучалась лишь на части данных), что повысит ее обобщающую способность:

In [17]:
# функция для записи прогнозов в файл
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

**Обучите модель на всей выборке, сделайте прогноз для тестовой выборки и сделайте посылку в соревновании**.

In [18]:
logit = LogisticRegression(n_jobs=-1, C=1, random_state=17, solver='lbfgs')
logit.fit(X_train_sparse, y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=-1, penalty='l2', random_state=17,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [19]:
pred_probabilities = logit.predict_proba(X_test_sparse)
pred_probabilities[:, 1]

array([2.21954270e-03, 2.51892726e-09, 6.16010896e-09, ...,
       8.43147766e-03, 3.87735238e-04, 1.29530564e-05])

In [20]:
# series = pd.Series(data=pred_probabilities[:, 1], index=range(1, pred_probabilities.shape[0] + 1))
# series.to_csv(path_or_buf="../../data/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/benchmark_1.csv",
#               header=["target"],
#               index_label="session_id")

Если вы выполните эти действия и загрузите ответ на [странице](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования, то воспроизведете первый бенчмарк "Logit".

### 3. Улучшение модели, построение новых признаков

Создайте такой признак, который будет представлять собой число вида ГГГГММ от той даты, когда проходила сессия, например 201407 -- 2014 год и 7 месяц. Таким образом, мы будем учитывать помесячный [линейный тренд](http://people.duke.edu/~rnau/411trend.htm) за весь период предоставленных данных.

In [21]:
new_train_features = pd.DataFrame(index=train_df.index)
new_test_features = pd.DataFrame(index=test_df.index)

In [22]:
new_train_features.shape, new_test_features.shape

((253561, 0), (82797, 0))

In [23]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 21669 to 204762
Data columns (total 21 columns):
site1     253561 non-null int64
time1     253561 non-null datetime64[ns]
site2     253561 non-null int64
time2     250098 non-null datetime64[ns]
site3     253561 non-null int64
time3     246919 non-null datetime64[ns]
site4     253561 non-null int64
time4     244321 non-null datetime64[ns]
site5     253561 non-null int64
time5     241829 non-null datetime64[ns]
site6     253561 non-null int64
time6     239495 non-null datetime64[ns]
site7     253561 non-null int64
time7     237297 non-null datetime64[ns]
site8     253561 non-null int64
time8     235224 non-null datetime64[ns]
site9     253561 non-null int64
time9     233084 non-null datetime64[ns]
site10    253561 non-null int64
time10    231052 non-null datetime64[ns]
target    253561 non-null int64
dtypes: datetime64[ns](10), int64(11)
memory usage: 42.6 MB


In [24]:
time = ['time%d' % d for d in range(1, 11)]
train_na_zero = train_df[time].fillna(value=pd.Timestamp(0), inplace=False)
test_na_zero = test_df[time].fillna(value=pd.Timestamp(0), inplace=False)

In [25]:
train_na_zero.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 21669 to 204762
Data columns (total 10 columns):
time1     253561 non-null datetime64[ns]
time2     253561 non-null datetime64[ns]
time3     253561 non-null datetime64[ns]
time4     253561 non-null datetime64[ns]
time5     253561 non-null datetime64[ns]
time6     253561 non-null datetime64[ns]
time7     253561 non-null datetime64[ns]
time8     253561 non-null datetime64[ns]
time9     253561 non-null datetime64[ns]
time10    253561 non-null datetime64[ns]
dtypes: datetime64[ns](10)
memory usage: 21.3 MB


In [26]:
# Рассмотрим только время начала каждой сессии - поле time1
new_train_features['year_month'] = train_na_zero['time1'].apply(lambda ts: str(ts.year * 100 + ts.month))
new_test_features['year_month'] = test_na_zero['time1'].apply(lambda ts: str(ts.year * 100 + ts.month))

new_train_features.head()

,year_month
session_id,
21669,201301
54843,201301
77292,201301
114021,201301
146670,201301


In [27]:
scaler = StandardScaler()
scaler.fit(new_train_features['year_month'].values.reshape(-1, 1))

new_train_features['year_month_scaled'] = scaler.transform(new_train_features['year_month'].values.reshape(-1, 1))
new_test_features['year_month_scaled'] = scaler.transform(new_test_features['year_month'].values.reshape(-1, 1))

In [28]:
new_train_features.head()

,year_month,year_month_scaled
session_id,,
21669,201301,-1.744405
54843,201301,-1.744405
77292,201301,-1.744405
114021,201301,-1.744405
146670,201301,-1.744405


Добавьте новый признак, предварительно отмасштабировав его с помощью `StandardScaler`, и снова посчитайте ROC AUC на отложенной выборке.

In [29]:
# Нужно объединить новые данные с существующими в разреженной матрице X_train_sparse
# Делаем это с помощью функции hstack из библиотеки scipy.sparse
# Требуется дополнительное явное преобразование в csr_matrix
X_train_sparse_new = csr_matrix(hstack([X_train_sparse,
                                new_train_features['year_month_scaled'].values.reshape(-1, 1)]))
X_train_sparse_new.shape # (253561, 48372)

# Аналогично для тестового набора
X_test_sparse_new = hstack([X_test_sparse,
                             new_test_features['year_month_scaled'].values.reshape(-1, 1)])
X_test_sparse_new.shape # (82797, 48372)
X_train_sparse.shape, X_train_sparse_new.shape # Добавился дополнительный признак в shape[1]

((253561, 48371), (253561, 48372))

In [30]:
%%time
get_auc_lr_valid(X=X_train_sparse_new, y=y_train)

CPU times: user 313 ms, sys: 98.8 ms, total: 412 ms
Wall time: 2.67 s


0.9198902054055882

**Добавьте два новых признака: start_hour и morning.**

Признак `start_hour` – это час в который началась сессия (от 0 до 23), а бинарный признак `morning` равен 1, если сессия началась утром и 0, если сессия началась позже (будем считать, что утро это если `start_hour равен` 11 или меньше).

**Посчитйте ROC AUC на отложенной выборке для выборки с:**
- сайтами, `start_month` и `start_hour`
- сайтами, `start_month` и `morning`
- сайтами, `start_month`, `start_hour` и `morning`

In [31]:
# start_month + start_hour
# Создадим похожую фичу для идентификации, в какое время суток предполагаемый человек начинает сессию
new_train_features['start_hour'] = train_na_zero['time1'].apply(lambda ts: str(ts.hour))
new_test_features['start_hour'] = test_na_zero['time1'].apply(lambda ts: str(ts.hour))

# Фича с учётом года и месяца работает хуже
# new_train_features['start_hour'] = train_na_zero['time1'].apply(lambda ts: str((ts.year * 100 + ts.month) * 100 + ts.hour))
# new_test_features['start_hour'] = test_na_zero['time1'].apply(lambda ts: str((ts.year * 100 + ts.month) * 100 + ts.hour))

# start_month + start_hour + morning
# Аналогичную операцию проделаем для признака morning
new_train_features['morning'] = train_na_zero['time1'].apply(lambda ts: int(ts.hour <= 11))
new_test_features['morning'] = test_na_zero['time1'].apply(lambda ts: int(ts.hour <= 11))

In [32]:
scaler = StandardScaler()
scaler.fit(new_train_features['start_hour'].values.reshape(-1, 1))

new_train_features['start_hour_scaled'] = scaler.transform(new_train_features['start_hour'].values.reshape(-1, 1))
new_test_features['start_hour_scaled'] = scaler.transform(new_test_features['start_hour'].values.reshape(-1, 1))

In [33]:
new_train_features.head()

,year_month,year_month_scaled,start_hour,morning,start_hour_scaled
session_id,,,,,
21669,201301,-1.744405,8,1,-1.357366
54843,201301,-1.744405,8,1,-1.357366
77292,201301,-1.744405,8,1,-1.357366
114021,201301,-1.744405,8,1,-1.357366
146670,201301,-1.744405,8,1,-1.357366


In [34]:
new_test_features.head()

,year_month,year_month_scaled,start_hour,morning,start_hour_scaled
session_id,,,,,
1,201410,0.822948,11,1,-0.407823
2,201407,0.752287,11,1,-0.407823
3,201412,0.870055,15,0,0.858234
4,201411,0.846501,10,1,-0.724338
5,201405,0.705179,15,0,0.858234


In [35]:
# Снова объединяем новые данные с существующими в разреженной матрице X_train_sparse_new
# Делаем это с помощью функции hstack из библиотеки scipy.sparse
# Требуется дополнительное явное преобразование в csr_matrix
X_train_sparse_new_2 = csr_matrix(hstack([X_train_sparse_new,
                                  new_train_features['start_hour_scaled'].values.reshape(-1, 1)]))

# Аналогично для тестового набора
X_test_sparse_new_2 = hstack([X_test_sparse_new,
                             new_test_features['start_hour_scaled'].values.reshape(-1, 1)])
X_train_sparse_new.shape, X_train_sparse_new_2.shape # Добавился дополнительный признак в shape[1]

((253561, 48372), (253561, 48373))

In [36]:
%%time
# Проверяем качество с 2-мя признаками по ROC AUC
get_auc_lr_valid(X=X_train_sparse_new_2, y=y_train)

CPU times: user 362 ms, sys: 95.2 ms, total: 457 ms
Wall time: 2.75 s


0.9573322845076919

In [37]:
# Добавим в разреженную матрицу ещё одну фичу morning с помощью того же hstack
X_train_sparse_new_3 = csr_matrix(hstack([X_train_sparse_new_2,
                                  new_train_features['morning'].values.reshape(-1, 1)]))

# Аналогично для тестового набора
X_test_sparse_new_3 = hstack([X_test_sparse_new_2,
                             new_test_features['morning'].values.reshape(-1, 1)])
X_train_sparse_new_2.shape, X_train_sparse_new_3.shape # Добавился дополнительный признак в shape[1]

((253561, 48373), (253561, 48374))

In [38]:
%%time
# Проверяем качество с 3-мя признаками по ROC AUC
get_auc_lr_valid(X=X_train_sparse_new_3, y=y_train)

CPU times: user 360 ms, sys: 99.4 ms, total: 459 ms
Wall time: 2.73 s


0.9585507820000507

In [39]:
# Обучим новую модель

logit = LogisticRegression(n_jobs=-1, C=1, random_state=17, solver='lbfgs')
logit.fit(X_train_sparse_new_3, y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=-1, penalty='l2', random_state=17,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [40]:
# Создадим файл посылки для Kaggle с учётом 3-х новых фич
# Предсказанные результаты
pred_probabilities_3 = logit.predict_proba(X_test_sparse_new_3)
pred_probabilities_3[:, 1]

array([5.26264685e-05, 6.26559747e-08, 2.43439679e-09, ...,
       2.04955019e-04, 6.29702900e-06, 2.71673195e-07])

In [41]:
# write_to_submission_file(pred_probabilities_3[:, 1],
#                          out_file = "../../data/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/benchmark_1_5.csv",
#                          target='target',
#                          index_label="session_id")

### 4. Подбор коэффицициента регуляризации

Итак, мы ввели признаки, которые улучшают качество нашей модели по сравнению с первым бейслайном. Можем ли мы добиться большего значения метрики? После того, как мы сформировали обучающую и тестовую выборки, почти всегда имеет смысл подобрать оптимальные гиперпараметры -- характеристики модели, которые не изменяются во время обучения. Например, на 3 неделе вы проходили решающие деревья, глубина дерева это гиперпараметр, а признак, по которому происходит ветвление и его значение -- нет. В используемой нами логистической регрессии веса каждого признака изменяются и во время обучения находится их оптимальные значения, а коэффициент регуляризации остается постоянным. Это тот гиперпараметр, который мы сейчас будем оптимизировать.

Посчитайте качество на отложенной выборке с коэффициентом регуляризации, который по умолчанию `C=1`:

In [42]:
from sklearn.metrics import accuracy_score

In [43]:
%%time
# наша целевая переменная на отложенной выборке (для тестирования)
get_accuracy_value(X=X_train_sparse_new_3, y=y_train)

CPU times: user 65.4 ms, sys: 66 ms, total: 131 ms
Wall time: 2.42 s


0.9908372661662438

In [44]:
%%time
# Проверяем качество с 3-мя признаками по ROC AUC - так проверяется на Kaggle
get_auc_lr_valid(X=X_train_sparse_new_3, y=y_train, C=1, ratio=0.9)

CPU times: user 68.5 ms, sys: 74.4 ms, total: 143 ms
Wall time: 2.87 s


0.9585507820000507

Постараемся побить этот результат за счет оптимизации коэффициента регуляризации. Возьмем набор возможных значений C и для каждого из них посчитаем значение метрики на отложенной выборке.

Найдите `C` из `np.logspace(-3, 1, 10)`, при котором ROC AUC на отложенной выборке максимален. 

In [45]:
reg_coefficient = np.logspace(-3, 1, 10)

result = (0, 0)
for c in reg_coefficient:
    local_result = get_auc_lr_valid(X=X_train_sparse_new_3, ratio=0.9, y=y_train, C=c)
    if result[0] < local_result:
        result = (local_result, c)

result_reg_coefficient = result[1]
result

(0.960868674591127, 0.1668100537200059)

Наконец, обучите модель с найденным оптимальным значением коэффициента регуляризации и с построенными признаками `start_hour`, `start_month` и `morning`. Если вы все сделали правильно и загрузите это решение, то повторите второй бенчмарк соревнования.

In [46]:
# Создадим файл посылки для Kaggle с учётом 3-х новых фич и коэффициента регуляризации
# Предсказанные результаты
logit = LogisticRegression(n_jobs=-1, C=result_reg_coefficient, random_state=17, solver='lbfgs')
logit.fit(X_train_sparse_new_3, y_train)

pred_probabilities_2_nd_benchmark = logit.predict_proba(X_test_sparse_new_3)
pred_probabilities_2_nd_benchmark[:, 1]

array([2.54857950e-04, 2.35772047e-06, 9.26844124e-07, ...,
       5.60875922e-04, 6.04939606e-05, 7.53602794e-06])

In [47]:
# write_to_submission_file(pred_probabilities_2_nd_benchmark[:, 1],
#                          out_file = "../../data/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/benchmark_2.csv",
#                          target='target',
#                          index_label="session_id")

### 5. А теперь попробуем подобрать ещё несколько фич

In [48]:
# Попробуем добавить фичу с количеством сайтов, просматриваемых за одну сессию sites_amount_per_session

amount_of_unique_sites_train = X_train_sparse.sum(axis=1)
amount_of_unique_sites_test = X_test_sparse.sum(axis=1)

new_train_features['sites_amount_per_session'] = amount_of_unique_sites_train
new_test_features['sites_amount_per_session'] = amount_of_unique_sites_test

new_train_features['short_session'] = amount_of_unique_sites_train < 10
new_test_features['short_session'] = amount_of_unique_sites_test < 10

new_train_features['short_session'] = new_train_features['short_session'].astype('int')
new_test_features['short_session'] = new_test_features['short_session'].astype('int')
new_train_features.head()

,year_month,year_month_scaled,start_hour,morning,start_hour_scaled,sites_amount_per_session,short_session
session_id,,,,,,,
21669,201301,-1.744405,8,1,-1.357366,2,1
54843,201301,-1.744405,8,1,-1.357366,4,1
77292,201301,-1.744405,8,1,-1.357366,10,0
114021,201301,-1.744405,8,1,-1.357366,10,0
146670,201301,-1.744405,8,1,-1.357366,10,0


In [49]:
scaler_5 = StandardScaler()
scaler_5.fit(new_train_features['sites_amount_per_session'].values.reshape(-1, 1))

new_train_features['sites_amount_per_session_scaled'] = scaler_5.transform(new_train_features['sites_amount_per_session'].values.reshape(-1, 1))
new_test_features['sites_amount_per_session_scaled'] = scaler_5.transform(new_test_features['sites_amount_per_session'].values.reshape(-1, 1)) 
new_train_features.head()

,year_month,year_month_scaled,start_hour,morning,start_hour_scaled,sites_amount_per_session,short_session,sites_amount_per_session_scaled
session_id,,,,,,,,
21669,201301,-1.744405,8,1,-1.357366,2,1,-4.328969
54843,201301,-1.744405,8,1,-1.357366,4,1,-3.177031
77292,201301,-1.744405,8,1,-1.357366,10,0,0.278784
114021,201301,-1.744405,8,1,-1.357366,10,0,0.278784
146670,201301,-1.744405,8,1,-1.357366,10,0,0.278784


In [50]:
# Добавим в разреженную матрицу ещё одну фичу morning с помощью того же hstack
X_train_sparse_new_4 = csr_matrix(hstack([X_train_sparse_new_3,
                                  new_train_features[['sites_amount_per_session_scaled', 'short_session']].values]))

# Аналогично для тестового набора
X_test_sparse_new_4 = hstack([X_test_sparse_new_3,
                             new_test_features[['sites_amount_per_session_scaled', 'short_session']].values])
X_train_sparse_new_3.shape, X_train_sparse_new_4.shape # Добавились дополнительные признаки в shape[1]

((253561, 48374), (253561, 48376))

In [51]:
%%time
# Проверяем качество с 3-мя признаками по ROC AUC - так проверяется на Kaggle
get_auc_lr_valid(X=X_train_sparse_new_4, y=y_train, C=result_reg_coefficient, ratio=0.9)

CPU times: user 375 ms, sys: 103 ms, total: 478 ms
Wall time: 2.84 s


0.9617147695421517

In [52]:
result_1 = (0, 0)
for c in reg_coefficient:
    local_result = get_auc_lr_valid(X=X_train_sparse_new_4, ratio=0.9, y=y_train, C=c)
    if result_1[0] < local_result:
        result_1 = (local_result, c)

result_reg_coefficient_1 = result_1[1]
result_1

(0.9620494336824589, 0.05994842503189409)

In [53]:
%%time
# Проверяем качество с 3-мя признаками по ROC AUC - так проверяется на Kaggle
get_auc_lr_valid(X=X_train_sparse_new_4, y=y_train, C=result_reg_coefficient_1, ratio=0.9)

CPU times: user 74.6 ms, sys: 73.7 ms, total: 148 ms
Wall time: 2.45 s


0.9620494336824589

In [54]:
logit = LogisticRegression(n_jobs=-1, C=result_reg_coefficient_1, random_state=17, solver='lbfgs')
logit.fit(X_train_sparse_new_4, y_train)

pred_probabilities_benchmark_2_1 = logit.predict_proba(X_test_sparse_new_4)
pred_probabilities_benchmark_2_1[:, 1]

# write_to_submission_file(pred_probabilities_benchmark_2_1[:, 1],
#                          out_file = "../../data/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/benchmark_2_1.csv",
#                          target='target',
#                          index_label="session_id")

array([5.06128982e-04, 1.03396300e-05, 1.31627787e-05, ...,
       9.07472535e-04, 1.53460865e-04, 3.15474427e-05])

In [55]:
new_train_zero_na_df = train_na_zero.copy(deep=True)
new_test_zero_na_df = train_na_zero.copy(deep=True)

In [56]:
for i in range(9, 0, -1):
    intermediate_train_df = new_train_zero_na_df[(new_train_zero_na_df['time10'] == pd.Timestamp(0)) & \
                                                 (new_train_zero_na_df['time%s' % i] != pd.Timestamp(0))]
    new_train_zero_na_df.loc[intermediate_train_df.index.values,
                             ['time10']] = intermediate_train_df['time%s' % i]
    
    intermediate_test_df = new_test_zero_na_df[(new_test_zero_na_df['time10'] == pd.Timestamp(0)) & \
                                               (new_test_zero_na_df['time%s' % i] != pd.Timestamp(0))]
    new_test_zero_na_df.loc[intermediate_test_df.index.values,
                            ['time10']] = intermediate_test_df['time%s' % i]
    
new_train_zero_na_df.head()

,time1,time2,time3,time4,time5,time6,time7,time8,time9,time10
session_id,,,,,,,,,,
21669,2013-01-12 08:05:57,2013-01-12 08:05:57,1970-01-01 00:00:00,1970-01-01 00:00:00,1970-01-01 00:00:00,1970-01-01 00:00:00,1970-01-01 00:00:00,1970-01-01 00:00:00,1970-01-01 00:00:00,2013-01-12 08:05:57
54843,2013-01-12 08:37:23,2013-01-12 08:37:23,2013-01-12 09:07:07,2013-01-12 09:07:09,1970-01-01 00:00:00,1970-01-01 00:00:00,1970-01-01 00:00:00,1970-01-01 00:00:00,1970-01-01 00:00:00,2013-01-12 09:07:09
77292,2013-01-12 08:50:13,2013-01-12 08:50:14,2013-01-12 08:50:15,2013-01-12 08:50:15,2013-01-12 08:50:16,2013-01-12 08:50:16,2013-01-12 08:50:16,2013-01-12 08:50:16,2013-01-12 08:50:17,2013-01-12 08:50:17
114021,2013-01-12 08:50:17,2013-01-12 08:50:17,2013-01-12 08:50:18,2013-01-12 08:50:18,2013-01-12 08:50:18,2013-01-12 08:50:18,2013-01-12 08:50:19,2013-01-12 08:50:19,2013-01-12 08:50:19,2013-01-12 08:50:20
146670,2013-01-12 08:50:20,2013-01-12 08:50:20,2013-01-12 08:50:20,2013-01-12 08:50:21,2013-01-12 08:50:21,2013-01-12 08:50:21,2013-01-12 08:50:21,2013-01-12 08:50:22,2013-01-12 08:50:22,2013-01-12 08:50:22


In [57]:
# Добавляем новую фичу - is_quick_launch, которая показываеет, были ли открыти все сайты в сессии в течение 5 минут
# или открывались один за другим в течение более долгого времени
new_train_features['sites_open_time_frame'] = (new_train_zero_na_df['time10'] - \
                                               new_train_zero_na_df['time1']).apply(lambda dt: dt.total_seconds() / 60.0)

new_test_features['sites_open_time_frame'] = (new_test_zero_na_df['time10'] - \
                                              new_test_zero_na_df['time1']).apply(lambda dt: dt.total_seconds() / 60.0)

new_train_features['is_quick_launch'] = (new_train_features['sites_open_time_frame'] < 2).astype(np.int)

new_test_features['is_quick_launch'] = (new_test_features['sites_open_time_frame'] < 2).astype(np.int)

new_train_features.head()

,year_month,year_month_scaled,start_hour,morning,start_hour_scaled,sites_amount_per_session,short_session,sites_amount_per_session_scaled,sites_open_time_frame,is_quick_launch
session_id,,,,,,,,,,
21669,201301,-1.744405,8,1,-1.357366,2,1,-4.328969,0.000000,1
54843,201301,-1.744405,8,1,-1.357366,4,1,-3.177031,29.766667,0
77292,201301,-1.744405,8,1,-1.357366,10,0,0.278784,0.066667,1
114021,201301,-1.744405,8,1,-1.357366,10,0,0.278784,0.050000,1
146670,201301,-1.744405,8,1,-1.357366,10,0,0.278784,0.033333,1


In [58]:
scaler_6 = StandardScaler()
scaler_6.fit(new_train_features['sites_open_time_frame'].values.reshape(-1, 1))

new_train_features['sites_open_time_frame_scaled'] = scaler_6.transform(new_train_features['sites_open_time_frame'].values.reshape(-1, 1))
new_test_features['sites_open_time_frame_scaled'] = scaler_6.transform(new_test_features['sites_open_time_frame'].values.reshape(-1, 1)) 
new_train_features.head()

,year_month,year_month_scaled,start_hour,morning,start_hour_scaled,sites_amount_per_session,short_session,sites_amount_per_session_scaled,sites_open_time_frame,is_quick_launch,sites_open_time_frame_scaled
session_id,,,,,,,,,,,
21669,201301,-1.744405,8,1,-1.357366,2,1,-4.328969,0.000000,1,-0.468233
54843,201301,-1.744405,8,1,-1.357366,4,1,-3.177031,29.766667,0,5.570015
77292,201301,-1.744405,8,1,-1.357366,10,0,0.278784,0.066667,1,-0.454709
114021,201301,-1.744405,8,1,-1.357366,10,0,0.278784,0.050000,1,-0.458090
146670,201301,-1.744405,8,1,-1.357366,10,0,0.278784,0.033333,1,-0.461471


In [59]:
# Добавим в разреженную матрицу ещё одну фичу is_quick_launch с помощью того же hstack
X_train_sparse_new_5 = csr_matrix(hstack([X_train_sparse_new_4,
                                  new_train_features[['is_quick_launch']].values]))

# Аналогично для тестового набора
X_test_sparse_new_5 = hstack([X_test_sparse_new_4,
                             new_test_features[['is_quick_launch']].values])
X_train_sparse_new_4.shape, X_train_sparse_new_5.shape # Добавились дополнительные признаки в shape[1]

((253561, 48376), (253561, 48377))

In [60]:
result_2 = (0, 0)
for c in reg_coefficient:
    local_result = get_auc_lr_valid(X=X_train_sparse_new_5, ratio=0.9, y=y_train, C=c)
    if result_2[0] < local_result:
        result_2 = (local_result, c)

result_reg_coefficient_2 = result_2[1]
result_2

(0.9623047717038073, 0.05994842503189409)

In [61]:
%%time
# Проверяем качество с 8-мью признаками по ROC AUC - так проверяется на Kaggle
get_auc_lr_valid(X=X_train_sparse_new_5, y=y_train, C=result_reg_coefficient_2, ratio=0.9)

CPU times: user 75.8 ms, sys: 77 ms, total: 153 ms
Wall time: 2.52 s


0.9623047717038073

In [62]:
logit = LogisticRegression(n_jobs=-1, C=result_reg_coefficient_2, random_state=17, solver='lbfgs')
logit.fit(X_train_sparse_new_5, y_train)

pred_probabilities_benchmark_2_1_1 = logit.predict_proba(X_test_sparse_new_5)
pred_probabilities_benchmark_2_1_1[:, 1]

# write_to_submission_file(pred_probabilities_benchmark_2_1_1[:, 1],
#                          out_file = "../../data/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/benchmark_2_1_1.csv",
#                          target='target',
#                          index_label="session_id")

array([5.36124750e-04, 1.20388660e-05, 1.77178873e-05, ...,
       9.53617329e-04, 1.69406166e-04, 2.55158060e-05])

### 6. Попробуем применить подход TF-IDF

In [63]:
import scipy as sci

In [64]:
# табличка с индексами посещенных сайтов в сессии
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


In [65]:
# Разреженная матрица с мерами Term Frequency
tf_csr_matrix = np.dot(full_sites_sparse, 1 / full_sites_sparse.shape[1])
# print(tf_csr_matrix)

# Вектор количества упомининий каждого сайта во всех сессиях
overall_freq_vector = np.sum(full_sites_sparse > 0, axis=0)
overall_freq_vector = np.array(overall_freq_vector)
overall_freq_vector

array([[ 3725,   389, 15596, ...,     1,     5,     1]], dtype=int64)

In [66]:
# Создадим разреженную матрицу inverse document frequency:
inverse_document_frequency_vector = np.log10(np.divide(full_sites_sparse.shape[0], overall_freq_vector))
inverse_document_frequency_vector[inverse_document_frequency_vector == np.inf] = 0
inverse_document_frequency_vector_csr = sci.sparse.csr_matrix(inverse_document_frequency_vector)
inverse_document_frequency_vector_csr

<1x48371 sparse matrix of type '<class 'numpy.float64'>'
	with 48371 stored elements in Compressed Sparse Row format>

In [67]:
# inverse_document_frequency_matrix = sci.sparse.csr_matrix(np.diag(list(inverse_document_frequency_vector[0])))
# inverse_document_frequency_matrix

In [68]:
# Матрица TF-IDF
tf_idf_matrix_sparse = tf_csr_matrix.multiply(inverse_document_frequency_vector_csr)
tf_idf_matrix_sparse

<336358x48371 sparse matrix of type '<class 'numpy.float64'>'
	with 1866898 stored elements in Compressed Sparse Row format>

In [69]:
X_train_sparse_tf_idf = tf_idf_matrix_sparse[:idx_split]
X_test_sparse_tf_idf = tf_idf_matrix_sparse[idx_split:]
X_train_sparse_tf_idf.shape, y_train.shape

((253561, 48371), (253561,))

In [197]:
print(X_train_sparse_tf_idf)

  (0, 54)	2.6020656689760575e-05
  (0, 55)	2.7304207846449522e-05
  (1, 54)	5.204131337952115e-05
  (1, 55)	5.4608415692899044e-05
  (2, 948)	7.988348245434722e-05
  (2, 783)	3.70156695866675e-05
  (2, 947)	4.792872599372011e-05
  (2, 944)	7.458443424995822e-05
  (2, 950)	6.71486667740583e-05
  (2, 945)	0.0003250692273165672
  (3, 946)	6.869160817069894e-05
  (3, 945)	0.0001950415363899403
  (3, 948)	7.988348245434722e-05
  (3, 947)	9.585745198744022e-05
  (3, 944)	0.00022375330274987466
  (4, 950)	6.71486667740583e-05
  (4, 945)	0.00013002769092662687
  (4, 951)	7.950136495224857e-05
  (4, 947)	4.792872599372011e-05
  (4, 949)	0.0001593807820200155
  (4, 946)	0.00020607482451209683
  (5, 954)	7.811733754676568e-05
  (5, 945)	0.0001950415363899403
  (5, 952)	9.769268650998951e-05
  (5, 946)	0.00020607482451209683
  :	:
  (253557, 3345)	0.00010162593908741729
  (253557, 49)	3.491550343982676e-05
  (253557, 51)	2.306068331992799e-05
  (253557, 752)	9.286538765322759e-05
  (253557, 4206)	

In [70]:
%%time
get_auc_lr_valid(X=X_train_sparse_tf_idf, ratio=0.9, y=y_train, C=0.001)

CPU times: user 107 ms, sys: 71.6 ms, total: 178 ms
Wall time: 1.38 s


0.892248940003792

In [91]:
# scaler = StandardScaler(with_mean=False)
max_abs_scaler = MaxAbsScaler()
max_abs_scaler.fit(X_train_sparse_tf_idf)

X_train_sparse_tf_idf_1 = max_abs_scaler.transform(X_train_sparse_tf_idf)
X_test_sparse_tf_idf_1 = max_abs_scaler.transform(X_test_sparse_tf_idf)

In [174]:
# print(X_train_sparse_tf_idf_1)

  (0, 54)	0.10000000000000002
  (0, 55)	0.3333333333333333
  (1, 54)	0.20000000000000004
  (1, 55)	0.6666666666666666
  (2, 948)	0.5
  (2, 783)	0.16666666666666669
  (2, 947)	0.16666666666666666
  (2, 944)	0.25
  (2, 950)	0.09999999999999998
  (2, 945)	0.49999999999999994
  (3, 946)	0.2
  (3, 945)	0.29999999999999993
  (3, 948)	0.5
  (3, 947)	0.3333333333333333
  (3, 944)	0.75
  (4, 950)	0.09999999999999998
  (4, 945)	0.19999999999999998
  (4, 951)	0.5
  (4, 947)	0.16666666666666666
  (4, 949)	0.5
  (4, 946)	0.6000000000000001
  (5, 954)	0.5
  (5, 945)	0.29999999999999993
  (5, 952)	0.2
  (5, 946)	0.6000000000000001
  :	:
  (253557, 3345)	0.2222222222222223
  (253557, 49)	0.19999999999999998
  (253557, 51)	0.09999999999999998
  (253557, 752)	0.6666666666666666
  (253557, 4206)	0.6666666666666666
  (253558, 3328)	0.33333333333333337
  (253558, 3593)	0.5
  (253558, 7329)	1.0
  (253558, 3323)	0.2
  (253558, 978)	0.25
  (253558, 783)	0.33333333333333337
  (253558, 3345)	0.2222222222222223


In [86]:
%%time
get_auc_lr_valid(X=X_train_sparse_tf_idf_1, ratio=0.9, y=y_train, C=0.001)

CPU times: user 66.7 ms, sys: 314 ms, total: 380 ms
Wall time: 1.52 s


0.8929964622513341

In [87]:
# Добавим в разреженную матрицу tf-idf все фичи, что мы делали раньше с помощью того же hstack
X_train_sparse_tf_idf_2 = csr_matrix(hstack([X_train_sparse_tf_idf_1,
                                     new_train_features[['is_quick_launch',
                                                         'sites_amount_per_session_scaled',
                                                         'short_session',
                                                         'start_hour_scaled',
                                                         'morning',
                                                         'year_month_scaled']].values]))

# Аналогично для тестового набора
X_test_sparse_tf_idf_2 = csr_matrix(hstack([X_test_sparse_tf_idf_1,
                                    new_test_features[['is_quick_launch',
                                                       'sites_amount_per_session_scaled',
                                                       'short_session',
                                                       'start_hour_scaled',
                                                       'morning',
                                                       'year_month_scaled']].values]))
X_train_sparse_tf_idf_1.shape, X_train_sparse_tf_idf_2.shape # Добавились дополнительные признаки в shape[1]

((253561, 48371), (253561, 48377))

In [103]:
result_3 = (0, 0)
for c in reg_coefficient:
    local_result = get_auc_lr_valid(X=X_train_sparse_tf_idf_2, ratio=0.9, y=y_train, C=c)
    if result_3[0] < local_result:
        result_3 = (local_result, c)

result_reg_coefficient_3 = result_3[1]
result_3

/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 6.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 6.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 6.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 6.
  " = {}.".format(

(0.9618295697580757, 1.2915496650148828)

In [89]:
logit = LogisticRegression(n_jobs=-1, C=result_reg_coefficient_3, random_state=17, solver='lbfgs')
logit.fit(X_train_sparse_tf_idf_2, y_train)

pred_probabilities_benchmark_2_1_1_tf_idf = logit.predict_proba(X_test_sparse_tf_idf_2)
pred_probabilities_benchmark_2_1_1_tf_idf[:, 1].shape

(82797,)

In [90]:
write_to_submission_file(pred_probabilities_benchmark_2_1_1_tf_idf[:, 1],
                         out_file = "../../data/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/benchmark_2_1_1_tf_idf.csv",
                         target='target',
                         index_label="session_id")

In [120]:
# Попробуем добавить фичи, связанные с временем суток (утро, день, вечер, ночь)
# Таким образом попробуем дифференцировать пользователей на несколько категорий
# Взято у yorko
def add_time_features(df, X_sparse):
    hour = df['time1'].apply(lambda ts: ts.hour)
    morning = ((hour >= 7) & (hour <= 11)).astype('int')
    day = ((hour >= 12) & (hour <= 18)).astype('int')
    evening = ((hour >= 19) & (hour <= 23)).astype('int')
    night = ((hour >= 0) & (hour <= 6)).astype('int')
    X = hstack([X_sparse, morning.values.reshape(-1, 1), 
                day.values.reshape(-1, 1), evening.values.reshape(-1, 1), 
                night.values.reshape(-1, 1)])
    return X

In [116]:
logit = LogisticRegression(C=1, random_state=17, solver='liblinear')

In [117]:
time_split = TimeSeriesSplit(n_splits=10)

In [118]:
%%time

# hangs with n_jobs > 1, and locally this runs much faster
cv_scores = cross_val_score(logit, X_train, y_train, cv=time_split, 
                            scoring='roc_auc', n_jobs=-1)

cv_scores, cv_scores.mean()

CPU times: user 2min 21s, sys: 6.46 s, total: 2min 27s
Wall time: 28.2 s


In [127]:
%%time
X_train_sparse_new_6 = add_time_features(train_df.fillna(0), X_train)
X_test_sparse_new_6 = add_time_features(test_df.fillna(0), X_test)

X_train_sparse_new_6.shape, X_test_sparse_new_6.shape

CPU times: user 1min 26s, sys: 1.77 s, total: 1min 28s
Wall time: 1min 24s


In [130]:
%%time
# hangs with n_jobs > 1, and locally this runs much faster
cv_scores = cross_val_score(logit, X_train_sparse_new_6, y_train, cv=time_split, 
                            scoring='roc_auc', n_jobs=-1)

CPU times: user 844 ms, sys: 142 ms, total: 986 ms
Wall time: 22.3 s


In [131]:
cv_scores, cv_scores.mean()

(array([0.87652264, 0.75129589, 0.93062182, 0.97864183, 0.90399565,
        0.9383148 , 0.96249244, 0.92731279, 0.94886477, 0.9404352 ]),
 0.915849783443105)

In [161]:
# Добавим дополнительные фичи
X_train_sparse_new_7 = csr_matrix(hstack([X_train_sparse_new_6,
                                     new_train_features[[
                                                         'is_quick_launch', # good idea
#                                                          'short_session', # bad idea
#                                                          'sites_amount_per_session_scaled', # bad idea
#                                                          'start_hour_scaled', # bad idea
                                                         'year_month_scaled'
                                     ]].values]))

# Аналогично для тестового набора
X_test_sparse_new_7 = csr_matrix(hstack([X_test_sparse_new_6,
                                    new_test_features[[
                                                       'is_quick_launch', # good idea
#                                                        'short_session', # bad idea
#                                                        'sites_amount_per_session_scaled', # bad idea
#                                                        'start_hour_scaled', # bad idea
                                                       'year_month_scaled'
                                    ]].values]))

In [162]:
X_train_sparse_new_7.shape, X_test_sparse_new_7.shape

((253561, 50006), (82797, 50006))

In [163]:
%%time
# hangs with n_jobs > 1, and locally this runs much faster
cv_scores = cross_val_score(logit, X_train_sparse_new_7, y_train, cv=time_split, 
                            scoring='roc_auc', n_jobs=-1)
cv_scores, cv_scores.mean()

CPU times: user 94.4 ms, sys: 129 ms, total: 223 ms
Wall time: 21.5 s


(array([0.88245795, 0.75922897, 0.94971744, 0.97923451, 0.90502538,
        0.94129751, 0.96494157, 0.92797815, 0.95043858, 0.94133002]),
 0.9201650083462501)

**Now we tune regularization parameter `C`.**

In [168]:
c_values = np.logspace(-2, 2, 10)

logit_grid_searcher = GridSearchCV(estimator=logit, param_grid={'C': c_values},
                                  scoring='roc_auc', n_jobs=-1, cv=time_split, verbose=1)

In [169]:
%%time
logit_grid_searcher.fit(X_train_sparse_new_7, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:   28.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  5.7min finished


CPU times: user 16.7 s, sys: 884 ms, total: 17.6 s
Wall time: 5min 45s


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
             error_score='raise-deprecating',
             estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=17, solver='liblinear',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': array([1.00000000e-02, 2.78255940e-02, 7.74263683e-02, 2.15443469e-01,
       5.99484250e-01, 1.66810054e+00, 4.64158883e+00, 1.29154967e+01,
       3.59381366e+01, 1.00000000e+02])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=

In [170]:
logit_grid_searcher.best_score_, logit_grid_searcher.best_params_

(0.9223720808549573, {'C': 0.21544346900318834})

In [171]:
logit_test_pred = logit_grid_searcher.predict_proba(X_test_sparse_new_7)[:, 1] #ROC AUC 0.9223720808549573
write_to_submission_file(logit_test_pred,
                         out_file = "../../data/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/benchmark_2_3.csv",
                         target='target',
                         index_label="session_id")

## 7. Ещё раз пробуем TF-IDF, но уже встроенный, сразу с оптимизацией

In [175]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [178]:
%%time
tf_idf_vectoriver = TfidfVectorizer()
with open('train_sessions_text.txt') as inp_train_file:
    X_train_tf_idf = tf_idf_vectoriver.fit_transform(inp_train_file)
with open('test_sessions_text.txt') as inp_test_file:
    X_test_tf_idf = tf_idf_vectoriver.transform(inp_test_file)
X_train_tf_idf.shape, X_test_tf_idf.shape

CPU times: user 3.23 s, sys: 46.6 ms, total: 3.28 s
Wall time: 2.33 s


((253561, 41592), (82797, 41592))

In [198]:
# print(X_train_tf_idf)

In [180]:
logit_tf_idf = LogisticRegression(C=1, random_state=17, solver='liblinear')

In [181]:
%%time
# hangs with n_jobs > 1, and locally this runs much faster
tf_idf_scores = cross_val_score(logit_tf_idf,
                                X_train_tf_idf,
                                y_train,
                                cv=time_split, 
                                scoring='roc_auc',
                                n_jobs=-1)
tf_idf_scores, tf_idf_scores.mean()

CPU times: user 83.1 ms, sys: 109 ms, total: 192 ms
Wall time: 4.04 s


(array([0.79738066, 0.66527936, 0.87207069, 0.93336481, 0.84811885,
        0.88257794, 0.92296525, 0.86409586, 0.92640888, 0.91885051]),
 0.863111282374818)

In [201]:
%%time
# Сравним значения с самодельным экземпляром TF-IDF vectorizer'а в матрице X_train_sparse_tf_idf_1
# hangs with n_jobs > 1, and locally this runs much faster
tf_idf_scores_test = cross_val_score(logit_tf_idf,
                                X_train_sparse_tf_idf_1,
                                y_train,
                                cv=time_split, 
                                scoring='roc_auc',
                                n_jobs=-1)
tf_idf_scores_test, tf_idf_scores_test.mean()
# Значение меньше, вероятно потому, что матрица была дополнительно нормализована, к примеру.

CPU times: user 86.3 ms, sys: 124 ms, total: 211 ms
Wall time: 4.22 s


(array([0.75953502, 0.6187712 , 0.85340736, 0.92646867, 0.82223631,
        0.86798529, 0.91439287, 0.87419429, 0.91275137, 0.9145681 ]),
 0.8464310472287077)

In [182]:
%%time
X_train_tf_idf_1 = add_time_features(train_df.fillna(0), X_train_tf_idf)
X_test_tf_idf_1 = add_time_features(test_df.fillna(0), X_test_tf_idf)

X_train_tf_idf_1.shape, X_test_tf_idf_1.shape

CPU times: user 1min 28s, sys: 1.83 s, total: 1min 30s
Wall time: 1min 28s


((253561, 41596), (82797, 41596))

In [183]:
%%time
# hangs with n_jobs > 1, and locally this runs much faster
tf_idf_scores = cross_val_score(logit_tf_idf,
                                X_train_tf_idf_1,
                                y_train,
                                cv=time_split, 
                                scoring='roc_auc',
                                n_jobs=-1)
tf_idf_scores, tf_idf_scores.mean()

CPU times: user 390 ms, sys: 118 ms, total: 508 ms
Wall time: 5.14 s


(array([0.8671512 , 0.80556054, 0.92529226, 0.96421019, 0.91430846,
        0.94855961, 0.94764803, 0.93213002, 0.9518587 , 0.94846402]),
 0.9205183037748885)

In [184]:
# Добавим дополнительные фичи
X_train_tf_idf_2 = csr_matrix(hstack([X_train_tf_idf_1,
                                     new_train_features[[
                                         'is_quick_launch',
                                         'year_month_scaled'
                                     ]].values]))

# Аналогично для тестового набора
X_test_tf_idf_2 = csr_matrix(hstack([X_test_tf_idf_1,
                                    new_test_features[[
                                         'is_quick_launch',
                                         'year_month_scaled'
                                    ]].values]))

In [186]:
%%time
# hangs with n_jobs > 1, and locally this runs much faster
tf_idf_scores = cross_val_score(logit_tf_idf,
                                X_train_tf_idf_2,
                                y_train,
                                cv=time_split, 
                                scoring='roc_auc',
                                n_jobs=-1)
tf_idf_scores, tf_idf_scores.mean()

CPU times: user 91.5 ms, sys: 120 ms, total: 212 ms
Wall time: 6.04 s


(array([0.86712327, 0.81176322, 0.94944271, 0.96540641, 0.91616295,
        0.95040444, 0.95195513, 0.93446082, 0.95274383, 0.94995003]),
 0.9249412808323381)

In [192]:
c_values = np.logspace(-2, 2, 10)

logit_grid_searcher_tf_idf = GridSearchCV(estimator=logit_tf_idf,
                                          param_grid={'C': c_values},
                                          scoring='roc_auc',
                                          n_jobs=-1,
                                          cv=time_split,
                                          verbose=1)

In [193]:
%%time
logit_grid_searcher_tf_idf.fit(X_train_tf_idf_2, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.0min finished


CPU times: user 11.4 s, sys: 682 ms, total: 12.1 s
Wall time: 1min 2s


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
             error_score='raise-deprecating',
             estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=17, solver='liblinear',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': array([1.00000000e-02, 2.78255940e-02, 7.74263683e-02, 2.15443469e-01,
       5.99484250e-01, 1.66810054e+00, 4.64158883e+00, 1.29154967e+01,
       3.59381366e+01, 1.00000000e+02])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=

In [194]:
logit_grid_searcher_tf_idf.best_score_, logit_grid_searcher_tf_idf.best_params_

(0.9268835730331638, {'C': 4.6415888336127775})

In [195]:
logit_test_pred_tf_idf = logit_grid_searcher_tf_idf.predict_proba(X_test_tf_idf_2)[:, 1] #ROC AUC 0.9268835730331638
write_to_submission_file(logit_test_pred_tf_idf,
                         out_file = "../../data/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/benchmark_2_4_tf_idf.csv",
                         target='target',
                         index_label="session_id")